In [1]:
# pip install hyperopt
from hyperopt import tpe, hp, fmin

# tpe -> algorithm 
# hp -> search space
# fmin -> minimize the equation

In [2]:
from sklearn import datasets

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

In [3]:
iris = datasets.load_iris()
x = iris.data
y = iris.target

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [5]:
def init_estimator(args):
    if args['model']==KNeighborsClassifier:
        estim = KNeighborsClassifier(
                                n_neighbors=args['param']['n_neighbors'],
                                algorithm=args['param']['algorithm'],
                                leaf_size=args['param']['leaf_size'],
                                metric=args['param']['metric']
        )

    elif args['model']==SVC:
        estim = SVC(
                C=args['param']['C'], 
                kernel=args['param']['kernel'], 
                degree=args['param']['degree'],
                gamma=args['param']['gamma']
        )

    elif args['model']==XGBClassifier:
        estim = XGBClassifier(
                      colsample_bytree=args['param']['colsample_bytree'],
                      gamma=args['param']['gamma'],
                      learning_rate=args['param']['learning_rate'],
                      subsample=args['param']['subsample'],
                      max_depth=int(args['param']['max_depth']), # DON'T FORGET INT()
                      n_estimators=int(args['param']['n_estimators']), # DON'T FORGET INT()
        )
        
    return estim

In [6]:
def objective_func(args):
    estim = init_estimator(args)
    
    estim.fit(x_train,y_train)
    
    y_pred_train = estim.predict(x_train)
    
    cv_score = cross_val_score(estimator=estim, X=x_train, y=y_train, cv=5,
                               #scoring='accuracy', # without it -> standard classifier's scoring
                               ).mean()
    
    loss = cv_score

    print(args['model'])
    print('{}{:.3f}'.format('Train Score:\t', estim.score(x_train,y_train)))
    print('{}{:.3f}'.format('CV Score:\t', cv_score))
    print('{}{:.3f}'.format('Test Score:\t', estim.score(x_test,y_test)))
    print('='*10)
    
    return loss

In [7]:
space = hp.choice('classifier', # This is the Label followeed by list of estimators
        [
            {'model': KNeighborsClassifier,
             'param': {
                       'n_neighbors' : hp.choice('n_neighbors_KNN', range(3,11)),
                       'algorithm'   : hp.choice('algorithm_KNN', ['ball_tree', 'kd_tree']),
                       'leaf_size'   : hp.choice('leaf_size_KNN', range(1,50)),
                       'metric'      : hp.choice('metric_KNN', ['euclidean', 'manhattan', 'chebyshev', 'minkowski'])
                      }
            },

            {'model': SVC,
             'param': {
                       'C'      : hp.lognormal('C_SVC', 0,1),
                       'kernel' : hp.choice('kernel_SVC', ['rbf', 'poly', 'rbf', 'sigmoid']),
                       'degree' : hp.choice('degree_SVC', range(1,15)),
                       'gamma'  : hp.uniform('gamma_SVC', 0.001,10000)
                      }
            },
            
            {'model': XGBClassifier,
             'param': {
                       'colsample_bytree' : hp.uniform('colsample_bytree_XGB', 0.3, 0.7),
                       'gamma' : hp.uniform('gamma_XGB', 0, 0.7), 
                       'learning_rate' : hp.uniform('learning_rate_XGB', 0.01, 0.5),
                       'subsample' : hp.uniform('subsample_XGB', 0.3, 0.7),
                       
                       'max_depth' : hp.quniform('max_depth_XGB', 2, 6, 1), # 1 stands for q
                       'n_estimators' : hp.quniform('n_estimators_XGB', 50, 150, 1)
                      }
            }
        ])
# Labels should have unique names (e.g. if we put simply 'gamma' in both SVC and XGB ->)

In [8]:
best_classifier = fmin(objective_func,
                       space,
                       algo=tpe.suggest,
                       max_evals=10)

<class 'sklearn.svm.classes.SVC'>                   
Train Score:	1.000                                  
CV Score:	0.367                                     
Test Score:	0.200                                   
<class 'sklearn.neighbors.classification.KNeighborsClassifier'>
Train Score:	0.992                                                           
CV Score:	0.966                                                              
Test Score:	0.967                                                            
<class 'sklearn.svm.classes.SVC'>                                            
Train Score:	0.367                                                           
CV Score:	0.367                                                              
Test Score:	0.200                                                            
<class 'xgboost.sklearn.XGBClassifier'>                                      
Train Score:	0.983                                                           
CV Score:	0.941       

In [9]:
print(best_classifier)

{'C_SVC': 1.7432426300284527, 'classifier': 1, 'degree_SVC': 5, 'gamma_SVC': 8450.238491617683, 'kernel_SVC': 0}
